#### RPA_day3 정리
python-docx : 파이썬으로 워드문서를 생성하기 위해서 사용 모듈   
워드문서 객체 생성 - Document  
제목 추가 - add_heading("제목", style)  
단락 추가 - add_paragraph("내용", style)  
텍스트 추가 - add_run(텍스트) 
표 추가 - add_table(rows, cols), 
테이블 객체의 모든 행이 저장된 속성 - rows  
테이블 객체의 모든 행이 저장된 속성으로 셀을 참조할 떄 - cells  
표 객체에 행을 추가 - add_row()
이미지 추가 - add_picture  
워드 문서 객체를 물리적인 파일로 생성 - save()


pywin32 : 워드문서, 엑셀, PPT 객체 => pdf로 변환   
1. application실행 - Dispatch("App이름")  
2. 실행된 application을 이용해서 문서를 open => 문서 객체 (Document, Presentation, Workbook)     
3. 워드문서객체.SaveAs('경로/파일이름.pdf', FileFormat=17)    
   엑셀문서객체.ExportAsFixedFormat(0, '경로/파일이름.pdf')  
   PPT문서객체.SaveAs('경로/파일이름.pdf', 32)    
4. 문서객체.Close()  
5. 실행된 application 종료 - Quit()


pdfkit - wkhtmltopdf 유틸리티를 실행시켜서 웹 문서를 pdf로 저장  
웹 문서, 로컬에 저장된 html문서, 문자열 내용을 pdf로 저장하는 메서드 :   
from_url(url, saved_path_filename, options=, configuration)
from_string()   
from_file()    

PDF 문서 병합 : PdfFileReader, PdfFileMerger.append(), write()  
PDF 문서 분리 : PdfFileReader, PdfFileWriter.addPage(PdfFileReader.getPage()), write()  

PDF 문서로부터 일부 페이지 제거(삭제) - pikepdf.open(), del pdf문서객체.pages[idx], pdf문서객체.save()  
 
PDF 문서 회전 - pikepdf.open()로부터 반환된 pdf문서객체.Rotate=90  
PDF 문서로부터 텍스트 추출  
PDF 문서로부터 이미지 추출   

python-pptx - 파이썬으로 PPT문서를 생성 모듈  
Presentation() - PPT문서객체 생성  
PPT문서객체에 슬라이드 추가 - PPT문서객체.slides.add_slide(레이아웃)  
PPT문서객체.slide_layouts - 슬라이드 종류 선택 속성   
슬라이드에 내용 추가 - slide객체.placeholder[idx].text  

### 웹 자동화

웹 자동화   
• selenium을 활용하여 브라우저 상의 html 내용을 분석하여 클릭, 글자 입력, 체크박스나 라디오버튼 선택, 페이지 스크롤, 페이지 이동, Frame 전환, 로딩 대기 등의 작업을 할 수 있다.   
• requests - http://docs.python-requests.org   
• BeautifulSoup - https://www.crummy.com/software/BeautifulSoup/bs4/doc/   

웹 자동화 - 정적 웹 문서 읽어오기   
• requests모듈.get() - requests.models.Response 반환   
Response.status_code 속성 - 서버의 응답 상태   
Response.encoding 속성 - 사용된 인코딩 확인 및 사용할 인코딩 지정   
Response.text 속성 - HTML 소스 문자열

In [1]:
import requests

url = 'http://naver.com'
r = requests.get(url)
print(type(r))
print(r.status_code)  #HTTP 상태 코드(숫자에 따라 나뉨)
print(r.encoding)  #r.encoding = '인코딩방식'
r.encoding = 'utf-8' # 필요한 인코딩으로 설정 가능(utf-8, euc-kr)
r.text[:300] # 일부만 확인
html = r.text
open('naver.html', 'w', encoding='utf-8').write(html)  #응답 내용을 html문서롤 저장

<class 'requests.models.Response'>
200
UTF-8


172317

웹 자동화 - 이진 파일 읽어오기   
• requests모듈.get() 을 이용하여 얻은 이미지를 이진 파일로 저장하려면     Response.content 속성을 사용

웹에서 이미지 가져와서 저장하기

In [1]:
import requests

url = 'https://abi-laboratory.pro/images/opensource1.png'
r = requests.get(url)

#r콘텐츠의 내용을 opensource,png에 적으라는 뜻, wb=바이너리 쓰기모드
open('opensource.png', 'wb').write(r.content) #파일로 저장하는 이름 적음

13984

웹에서 pdf 가져와서 저장하기 

In [2]:
url = 'http://elearning.kocw.net/document/lec/2011/Kicu/ChaJungSub/01.pdf'
r = requests.get(url)
open('01.pdf', 'wb').write(r.content)

370559

웹에서 mp3 가져와서 저장하기 

In [3]:
url = 'http://jubil2018.cafe24.com/2018/mp3/jubil36/01.mp3'
r = requests.get(url)
open('jubil36_01.mp3', 'wb').write(r.content)

12033448

텍스트는 Response.text 속성에 저장되고  
이진데이터(binary data)는 Response.content속성에 저장

웹 자동화 - 대용량 파일 다운로드 (대용량 파일은 chunk 단위로 조금씩 저정해야 한다.)   
웹 요청으로 받은 대용량 파일의 경우,(python 실행환경의 할당 메모리를 초과할 수 있음   
=> 시스템 장애 발생 가능성)   
.get(stream=True)로 설정하여 웹 요청 파일을 받으면서 동시에 처리 가능(버퍼 설정, 사용)    
(requests.get(stream=True)로 설정하여 파일을 내려 받으면서 동시에 처리할 수 있도록 하고, Response 객체의 iter_content()에 버퍼 크기를 지정하여 반복자를 실행하여 버퍼가 찰 때마다 조금씩 파일로 저장한다.  )

In [ ]:
#1기가바이트가 넘는 우분투 리눅스 설치 파일 다운로드,
#파일이 너무 커서 실행은 안함
import requests

r = requests.get('http://releases.ubuntu.com/18.04.1/ubuntu-18.04.1-desktop-amd64.iso', stream=True)

with open('ubuntu-18.04.1-desktop-amd64.iso', 'wb') as f:
    for chunk in r.iter_content(100000): # 100k 바이트 단위로 처리
        f.write(chunk)
        print('.', end='', flush=True) # 100k 바이트를 내려받을 때마다 ,진행상황을 화면에 표시
    print()

웹 자동화 - 웹 자료 수집 연습문제   
• http://www.weather.go.kr/weather/climate/past_cal.jsp   
• 과거 월 단위로 HTML 문서에 정보를 저장   
1. 관측지점 번호, 년, 월 변수를 설정   
2. URL을 구성한다.  
3. 웹 페이지를 읽어온다  
4. 파일 이름을 정한다.  
5. 문서를 저장한다.  

In [9]:
#지정 파일 형식 w_108_2020_1.html,...w_108_2020_12.html파일로 저장
#stn=관측지점번호 
#방법1
import requests

url = "http://www.weather.go.kr/weather/climate/past_cal.jsp"
months = [m for m in range(1,13)]
for month in months:
    params = {
        "stn":108,
        "yy":2020,
        "mm":month
    }
    r = requests.get(url, params=params)
    html = r.text
    open(f"html/w_108_2020_{month}.html","w").write(html)


In [11]:
#방법2
def get_page(stn, yy, mm):
    url = f"http://www.weather.go.kr/weather/climate/past_cal.jsp?stn={stn}&yy={yy}&mm={mm}" #단계2
    res = requests.get(url) #단계3
    return res.text

stn = 108 #단계1
yy = 2020 #단계2
for mm in range(1, 13): #단계1
    html = get_page(stn, yy, mm)  #단계2-3
    fpath = f"w_{stn}_{yy}_{mm}.html" #단계4
    open(fpath, 'w', encoding='euc-kr').write(html)  #단계5
    print(fpath, 'saved')

w_108_2020_1.html saved
w_108_2020_2.html saved
w_108_2020_3.html saved
w_108_2020_4.html saved
w_108_2020_5.html saved
w_108_2020_6.html saved
w_108_2020_7.html saved
w_108_2020_8.html saved
w_108_2020_9.html saved
w_108_2020_10.html saved
w_108_2020_11.html saved
w_108_2020_12.html saved


웹 자동화 - 최신 뉴스 RSS로 읽기   
• RSS(Rich Site Summary)는 자주 발표되는 소식을 정보 제공자가 표준 XML 문서로 요약하여 배포(feed)한다   
• 구독자는 제공자가 공개하는 주소에서 RSS 문서를 가져와서 읽으면 새로운 소식들을 확인할 수 있다   
• pip install feedparser   
• http://news.mk.co.kr/rss/rss.php   
• feedparser는 XML 자료를 알아서 해석하고 파이썬 사전 자료형으로 변환한 결과를 반환한다.   
• feed속성 - RSS 피드 자체에 대한 정보 제공    
• entries 속성 - 기사( 제목, 링크, 요약, 공개된 시간) 저장 속성

In [12]:
! pip install feedparser

  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6070 sha256=7560d9de637537c04b57773a46a2c8468f82ad9cafe533291f06bc408dfb8fa3
  Stored in directory: c:\users\fbvl\appdata\local\pip\cache\wheels\83\63\2f\117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built sgmllib3k


In [18]:
import feedparser


print(type(d))
print(d.keys())  #키 딕셔너리를 모두 출력
print(d['href'])   #rss 피드 주소값 = 하이퍼링크에 해당하는 딕셔너리만 가져오기
print(d.href)
print(d.feed.title)  #rss피드(자체)정보
#print(d.entries)  #제공되는 기사가 저장되어 있는 속성
article = d.entries[0]  #제공되는 첫 번째 기사
print(article['title'])  #첫 번째 기사의 제목 
print(article['link'])   #첫 번째 기사를 제공하는 웹 페이지 url
print(article['summary']) #첫 번째 기사의 요약된 내용
print(article['published'])


<class 'feedparser.util.FeedParserDict'>
dict_keys(['bozo', 'entries', 'feed', 'headers', 'etag', 'updated', 'updated_parsed', 'href', 'status', 'encoding', 'bozo_exception', 'version', 'namespaces'])
http://file.mk.co.kr/news/rss/rss_30100041.xml
http://file.mk.co.kr/news/rss/rss_30100041.xml
매일경제 : 경제·금융 신문기사
일주일만에 1300만원↓…비트코인 7000만원대 붕괴
https://www.mk.co.kr/news/economy/view/2021/04/377774/
정부가 이달부터 가상화폐 불법행위에 대한 특별단속에 나서겠다고 밝히자 대표적인 가상화폐인 비트코인 가격이 7000만원 아래로 밀렸다. 일주일 만에 1300만원이나 빠졌다.20일 오전 9시 20분 현재 국내 가상화폐 ..
2021-04-20T09:48:38+09:00


매일경제에서 제공하는 경제 카테고리에서 제공하는 기사중에서  
제목에 '코인'이 포함되어 있거나 기사내용 요약에 '코인'이 포함된 내용만 제목 전체와 기사를 제공하는 웹 페이지 url을 출력하세요

In [22]:
d = feedparser.parse('http://file.mk.co.kr/news/rss/rss_30100041.xml') # 경제 RSS

for entry in d.entries:
    if '코인' in entry['title'] or '코인' in entry['summary']:
        print('*', entry['title'], entry['link'])

* 일주일만에 1300만원↓…비트코인 7000만원대 붕괴 https://www.mk.co.kr/news/economy/view/2021/04/377774/
* '코인광풍' 더 못봐…정부, 대규모 자금 인출 추적한다 https://www.mk.co.kr/news/economy/view/2021/04/376730/
* "10분만에 천만원 날려"…코인판, 도박판이 따로없네 https://www.mk.co.kr/news/economy/view/2021/04/376709/
* "美재무부 돈세탁 조사"…트윗 한 줄에 15% 출렁 https://www.mk.co.kr/news/economy/view/2021/04/376707/
* "20년 기다리느니 주식 코인할래" 보험 깨는 서민들 https://www.mk.co.kr/news/economy/view/2021/04/376661/
* 국내 알트코인 시가총액, 올해 들어서만 5배로 https://www.mk.co.kr/news/economy/view/2021/04/375542/
* 비트코인 1000만원 폭락…"美 재무부 돈세탁 조사설" https://www.mk.co.kr/news/economy/view/2021/04/372845/
* 가상화폐 하루 24조 거래…가이드라인 나온다 https://www.mk.co.kr/news/economy/view/2021/04/372183/


웹 자동화 - HTML 문서 분석   
• BeautifulSoup( html문서, 파서) – HTML, XML 문서의 해석과 필요한 정보 추출을 도와주는 모듈   
• https://www.crummy.com/software/BeautifulSoup/bs4/doc/

웹 자동화 - HTML 문서 분석   

• prettify() - 들여쓰기를 적용해 보기 좋게 출력   

• BeautifulSoup 객체에서 하위 태그는 온점(.)을 이용하여 접근할 수 있다

• tag.attrs[’속성명’] - 태그 내의 속성을 참조할때 사용

• tag.text - 태그가 가지고 있는 하위 텍스트 문자열 반환

• find_next() - 어떤 노드의 다음 노드를 찾을 수 있다.

• find(태그, string=) – 하나의 노드를 검색, string 옵션을 지정하면 조건에 맞는 문자열을 포함하는 텍스트 정보 추출

• find_all(태그, string=) – 조건에 맞는 모든 노드 검색

In [37]:
html = """
<html>
<head>
<title> python </title>
<meta charset='utf-8'>
</head>
<body>
<H1 class='title'python 활용 범위</H1>>
<p class='story'>파이썬(Python)은 1991년 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로,
플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically) 대화형 언어이다.</p>
<p> 자바(Java) </p>
<div class='book python'>
<a href = "http://www.yes24.com/Product/Goods/74419916?OzSrank=1" 
class = "python" name="jump" id="link1">점프 투 파이썬 </a>
<a href = "http://www.yes24.com/Product/Goods/74269975?OzSrank=2" 
class = "python" name="alone" id="link2">혼자 공부하는 파이썬 파이썬 </a>
<a href = "http://www.yes24.com/Product/Goods/98135876?OzSrank=3" 
class = "python" name="coinsale" id="link3">파이썬을 이용한 비트코인 자동매매 </a>
<a href = "http://www.yes24.com/Product/Goods/960924871?OzSrank=4" 
class = "python" name="ml" id="link4">혼자하는 머신러닝+딥러닝 </a>
<a href = "http://www.yes24.com/Product/Goods/91433923?OzSrank=5" 
class = "python" name="codingtest" id="link1">이것이 취업을 위한 코딩 테스트다 with파이썬 </a>
<body>
</html>
"""

In [38]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'lxml')
print(soup)
print(type(soup))  #타입출력
print(soup.prettify())  #soup을 읽기 쉽게 나타내줌
print(soup.html.head.title, soup.title)  #html안에 head안에 title출력! 두가지 다 같은 결과가 나온다
print(soup.a) #soup의 첫 번째 하이퍼링크 출력

<html>
<head>
<title> python </title>
<meta charset="utf-8"/>
</head>
<body>
<h1 class="title" python="">&gt;
</h1><p class="story">파이썬(Python)은 1991년 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로,
플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically) 대화형 언어이다.</p>
<p> 자바 </p>
<div class="book python">
<a class="python" href="http://www.yes24.com/Product/Goods/74419916?OzSrank=1" id="link1" name="jump">점프 투 파이썬 </a>
<a class="python" href="http://www.yes24.com/Product/Goods/74269975?OzSrank=2" id="link2" name="alone">혼자 공부하는 파이썬 파이썬 </a>
<a class="python" href="http://www.yes24.com/Product/Goods/98135876?OzSrank=3" id="link3" name="coinsale">파이썬을 이용한 비트코인 자동매매 </a>
<a class="python" href="http://www.yes24.com/Product/Goods/960924871?OzSrank=4" id="link4" name="ml">혼자하는 머신러닝+딥러닝 </a>
<a class="python" href="http://www.yes24.com/Product/Goods/91433923?OzSrank=5" id="link1" name="codingtest">이것이 취업을 위한 코딩 테스트다 with파이썬 </a>
</div></body></html>
<class 'bs4.BeautifulSoup'>
<html>
 <head>
  <title>
   python
  <

In [39]:
tag = soup.a  #문서 내에서 첫 번째 <a> 반환
print(tag.name)  #name속성을 출력
print(tag.text)  #태그 내용
print(tag.attrs['href'])   #[]안에는 속성명, 태그의 특정 속성 반환
print(tag.attrs)   #태그의 모든 속성을 dict로 반환 

a
점프 투 파이썬 
http://www.yes24.com/Product/Goods/74419916?OzSrank=1
{'href': 'http://www.yes24.com/Product/Goods/74419916?OzSrank=1', 'class': ['python'], 'name': 'jump', 'id': 'link1'}


In [34]:
print(soup.a.find_next())  #어떤 노드의 다음 노드를 반환(tag의 다음 a반환)
print(soup.a.find_next().find_next)  #그 다음 노드 반환 

<a class="python" href="http://www.yes24.com/Product/Goods/74269975?OzSrank=2" id="link2" name="alone">혼자 공부하는 파이썬 파이썬 </a>
<bound method PageElement.find_next of <a class="python" href="http://www.yes24.com/Product/Goods/74269975?OzSrank=2" id="link2" name="alone">혼자 공부하는 파이썬 파이썬 </a>>


In [43]:
print(soup.find('h1'))   #soup.h1과 동일
print(soup.find('h1', 'title'))  #h1태그 중 클래스 이름이 title인 태그 검색
print(soup.find('h1', attrs={'class':'title'}))  #위와 동일한데 딕셔너리로 클래스를 나타낸 것

<h1 class="title" python="">&gt;
</h1>
<h1 class="title" python="">&gt;
</h1>
<h1 class="title" python="">&gt;
</h1>


In [57]:
import re
print(soup.find('p', string=re.compile('^파이썬')))  #파이썬이 들어간 하나의 태그를 가져옴
print(soup.find_all('p', string=re.compile('^파이썬'))) #파이썬이 들어간 모든 태그를 가져옴
print(soup.find_all('a'))

print(soup.find_all('a', attrs={'class':'python', 'name':'ml'}))
print(soup.find_all(['a', 'h1']))
print(soup.find_all())

<p class="story">파이썬(Python)은 1991년 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로,
플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically) 대화형 언어이다.</p>
<p class="story">파이썬(Python)은 1991년 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로,
플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically) 대화형 언어이다.</p>
[<a class="python" href="http://www.yes24.com/Product/Goods/74419916?OzSrank=1" id="link1" name="jump">점프 투 파이썬 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/74269975?OzSrank=2" id="link2" name="alone">혼자 공부하는 파이썬 파이썬 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/98135876?OzSrank=3" id="link3" name="coinsale">파이썬을 이용한 비트코인 자동매매 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/960924871?OzSrank=4" id="link4" name="ml">혼자하는 머신러닝+딥러닝 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/91433923?OzSrank=5" id="link1" name="codingtest">이것이 취업을 위한 코딩 테스트다 with파이썬 </a>]
[<a class="python" href="http://www.yes24.com/Product/Goods/960924871?OzSrank=4" id="link4" name="ml">혼자

CSS의 selector문법으로 html문서 내용을 검색해서 결과 반환   
'태그명', '태그명', ....   
'.class속성값'  
'#id값'  
'부모태그명 자손태그'  
'부모태그명 > 자식태그'   
'태그명[속성명 = 값]'   
'태그 + next태그'   
'태그 ~ next태그'   
:nth-of-type(3n+1)    --가상환경을 가져올 시

In [82]:
import requests
from bs4 import BeautifulSoup
import re
import bs4
from bs4 import Tag

print(soup.select('#link1'))
print(soup.select('a.python'))
print(soup.select('p, a'))
print(soup.select('div > a'))
print(soup.select('body > a'))  #결과가 없으면 빈 리스트를 반환 
print(soup.select('a[href^="http:"]'))   #http:로 시작하는 a 요소
print(soup.select('a:nth-of-type(1)'))  #부모안에 a라는 요소 중 N번째 요소
print(soup.select('a:nth-of-type(3)'))


for ele in soup.select('a[href^="http:"]'):
    print(ele.get_text(), ele.get('href'))

[<a class="python" href="http://www.yes24.com/Product/Goods/74419916?OzSrank=1" id="link1" name="jump">점프 투 파이썬 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/91433923?OzSrank=5" id="link1" name="codingtest">이것이 취업을 위한 코딩 테스트다 with파이썬 </a>]
[<a class="python" href="http://www.yes24.com/Product/Goods/74419916?OzSrank=1" id="link1" name="jump">점프 투 파이썬 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/74269975?OzSrank=2" id="link2" name="alone">혼자 공부하는 파이썬 파이썬 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/98135876?OzSrank=3" id="link3" name="coinsale">파이썬을 이용한 비트코인 자동매매 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/960924871?OzSrank=4" id="link4" name="ml">혼자하는 머신러닝+딥러닝 </a>, <a class="python" href="http://www.yes24.com/Product/Goods/91433923?OzSrank=5" id="link1" name="codingtest">이것이 취업을 위한 코딩 테스트다 with파이썬 </a>]
[<p class="story">파이썬(Python)은 1991년 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로,
플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(d

웹 자동화 - 네이버 검색 작업 자동화 연습문제    
• 원하는 질의어 이용 검색하고, 결과 중에서 포스트와 블로그 섹션의 검색 결과를 출력   
1. 네이버 질의어 형식에 맞춰 url 주소를 작성한다.  
형식 = (http://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf7&query=검색질의어)
2. Work 2에서 배운 requests 모듈을 이용하여 문서를 검색한다.   
3. 결과로 얻은 HTML 문자열을 BeautifulSoup 객체로 만든다.   
4. 게시물 제목을 포함하는 태그를 추출한다.   
5. 제목과 간단한 내용 그리고 링크를 추출하여 출력한다   

In [85]:
import webbrowser

search = '마라탕'
r= requests.get('http://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf7&query='+search)  #단계2
soup = BeautifulSoup(r.text, 'lxml')  #단계3
for i, anchor in enumerate(soup.select('a.api_txt_lines')):
    print(i +1, anchor.text, '-', anchor.attrs['href'])
    print()

1 노량진 마라탕 '마라공방' 마라탕+꿔바로우 너무 맛있어요 - https://blog.naver.com/jiya_v3/222276754979

2 마라탕 잘드세요? - https://cafe.naver.com/pajumom/3911194

3 순대전골1인분과 마라탕2인분 - https://cafe.naver.com/masanmam/5446737

4 마라탕 - https://cafe.naver.com/bongmammamia/354600

5 울산송정마라탕 맛집 라화공방 방역수칙 준수까지 최고 - https://blog.naver.com/pjybd0203/222310302239

6 교대역마라탕 맛집 - 교대역밥집 '마라청춘마라탕' - https://blog.naver.com/2hyeono/222304613657

7 [부천역 마라탕 맛집] 도미라에서 마라탕과 꿔바로우로 데이트 ^^ - https://blog.naver.com/c082300/222302216816

8 꽂힌건가"라며 "강원도가 마라탕과 짜장면에 고량주를 함께하는 고장이 안 되었으면 한다"고 날카롭게... (강원도가) 마라탕과 짜장면에 고량주가 함께하는 고장이 안 되었으면 한다"고 날을 세웠다. 여기에 덧붙여 이 전... - https://www.sedaily.com/NewsView/22L2GT7ND1

9 중국 쓰촨(四川) 지방 향신료인 '마라'를 넣고 끓인 중국 음식 마라탕(麻辣汤)이 최근 2, 3년간 한국에서 선풍적인 인기를 끌었다.마라탕은 지금도 배달 어플 중국음식 메뉴에서 상위랭킹을 차지하고 있는 인기 음식이다.... - https://www.wikitree.co.kr/articles/638474

10 같은 기간 매운 중국 향신료인 마라를 이용해 만든 ‘피코크 마라탕’ 매출도 47% 증가했다. 피코크 마라탕은 피코크 냉장국탕 매출 가운데 차돌된장찌개 사골육수 육개장에 이어 4위를 차지하기도 했다. 지난해 해당... - http://www.kookje.co.kr/news2011/asp/

웹 자동화 - 웹 페이지의 이미지 모두 내려받기 연습문제      
• 인테리어 집 꾸미기 서비스 홈페이지인 https://ohou.se의 첫 페이지에 올라온 일정 크기이상의 이미지들을 모두 내려받아 보기    
• data-src 속성을 갖는 <img> 이미지 태그를 찾는다   
• 이미지 주소와 가로/세로 정보를 추출   
• 가로 이미지 크기가 640 이상이고 jpg인 이미지만 선택하고  
• 파일 이름을 추출하고 이미지를 가져와서 파일에 저장한다.  

In [134]:
! pip install selenium

In [137]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

driver = webdriver.Chrome("./chromedriver")

driver.get("https://ohou.se/")
time.sleep(2)
html = driver.page_source
driver.close()

bs = BeautifulSoup(html)

images = bs.select("img")
for k, i in enumerate(images):
    res = requests.get(i.get("src"))
    img = res.content
    im = Image.open(BytesIO(img))
    w, h = im.size
    print(w, h, im.format.lower())
    if w >= 640 and im.format.lower() == "jpeg":
        im.save(f"image/{k}.jpg")


1 1 gif
720 480 jpeg
36 36 jpeg
512 969 png
258 180 png
258 180 png
258 180 png
258 180 png
258 180 png
258 180 png
258 180 png
258 180 png
1449 982 gif
36 36 jpeg
240 160 jpeg
36 36 jpeg
240 160 jpeg
36 36 png
144 144 png
144 144 png
144 144 png
144 144 png
144 144 png
144 144 jpeg
144 144 png
144 144 png
144 144 png
144 144 png
144 144 png
144 144 jpeg
144 144 jpeg
144 144 png
144 144 jpeg
144 144 png
360 360 jpeg
1080 1080 gif
360 360 jpeg
360 360 jpeg


#### 브라우저 자동화   
• Selenium - 개발자들을 위해 웹 브라우저를 이용하여 웹 테스트를 자동으로 하기 위해 만들어진 도구   

• from selenium import webdriver - 드라이버를 구동하려면 실행 탐색 경로에 웹 드라이버가 있어야 한다.   

• webdriver.브라우저(웹 드라이버 경로) - Driver객체 반환   

• Driver객체.close() - 테스트 브라우저를 닫는다    

• ChromeOptions의 add_argument() - 브라우저를 실행시킬 때 옵션 설정 (옵션은 웹 드라이버마다 설정 방법이 다르다)

브라우저 자동화 – 요소를 찾는 메서드   

• Driver객체.find_element(s)_by_class_name( ) - class name을 갖는 태그 검색 , 실패하면 예외 발생, WebElement 객체 반환   

• Driver객체.find_element(s)_by_class_selector(selector ) - CSS selector와 일치하는 요소 검색   

• Driver객체.find_element(s)_by_xpath( ) - xpath 언어로 갖는 태그 검색 

• Driver객체.find_element_by_id(id ) - 태그의 id 속성값과 일치하는 요소 검색

• Driver객체.find_element(s)_by_link_text(text ) - 태그의 링크 텍스트가 text인 요소 검색

• Driver객체.find_element(s)_by_partial_link_text( ) - 태그의 링크 텍스트의 일부가 text인 요소 검색

• Driver객체.find_element(s)_by_name( name ) – name 속성의 값과 일치하는 요소 검색

• Driver객체.find_element(s)_by_tag_name( name ) – 태그 이름이 name인 요소 검색

In [183]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


#driver = webdriver.Chrome()  #크롬 브라우저 실행

options = webdriver.ChromeOptions()
options.add_argument('--disable-extensions')
options.add_argument("--start-maximized") # 최대 크기 윈도우로 시작
# options.add_argument('window-size=1200x600') # 윈도우 크기 지정
# options.headless = True # 윈도우가 눈에 보이지 않게 실행
# 실행 파일이 현재 경로에 없다면 executable_path로 지정해줘야 한다.
driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')

driver.get("http://www.python.org")
print(driver.title)
elem = driver.find_element_by_name("q")
print(elem)
print(elem.get_attribute('id'))
print(elem.get_attribute('name'))
print(elem.get_attribute('class'))
print(elem.tag_name)
print(elem.text)


Welcome to Python.org
<selenium.webdriver.remote.webelement.WebElement (session="a2e094c1c34204432ec7ca9f01acc34a", element="6eb271f9-ae41-4196-b39b-46321d9985f4")>
id-search-field
q
search-field
input



In [184]:
elem.send_keys("pycon" + Keys.RETURN)  #검색어와 검색요청
html = driver.page_source
print(html[:100])

#driver.find_elements_by_css_selector('.list-recent-events')
for tag in driver.find_elements_by_css_selector('.list-recent-events li h3'):
    print(tag.text, '-', tag.find_element_by_tag_name('a').get_attribute('href'))

for tag in driver.find_elements_by_xpath('//form/ul/li/h3'):
     print(tag.text, '-', tag.find_element_by_tag_name('a').get_attribute('href'))

    
elem = driver.find_elements_by_xpath('//form/ul/li/h3')  #어떤 조건으로 검색한 노드를 얻음
#for tag in elem.find_elements_by_xpath('ancestor-or-self::*'):  #조상 노드들을 출력
#    print(tag.tag_name)
    
png = driver.get_screenshot_as_png()
open('screen_capture.png', 'wb').write(png)

<html class="js no-touch geolocation fontface generatedcontent svg formvalidation placeholder boxsiz
PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
Conferences and Workshops - https://www.python.org/community/workshops
Event – PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
2008-04-14 PSF Board Meeting Minutes - https://www.python.org/psf/records/board/minutes/2008-04-14
2012-07-16 PSF Board Meeting Minutes - https://www.python.org/psf/records/board/minutes/2012-07-16
2013-02-06 PSF Board Meeting Minutes - https://www.python.org/psf/records/board/minutes/2013-02-06
PyCon Home at python.org - https://www.python.org/community/pycon
Event – PyCon Australia 2013 - https://www.python.org/events/python-events/57/
Event – PyCon AU 2019 - https://www.python.org/events/python-events/776/
Event – PyCon Australia 2014 - https://www.python.org/events/python-events/10/
Event – PyCon Ireland 2012 - https://www.python.org/events/pytho

130605

브라우저 자동화   
• Driver객체.get_screenshot_as_png() - 화면을 캡처   
• Driver객체.execute_scritp() - 웹 화면을 스크롤, window.scrollTo(0, 100) 과 같은 자바스크립트 명령을 수행해야 한다   
• Driver객체.window_handles[0] - 윈도우 핸들 , 팝업 창을 닫기 위해 사용   
• Driver객체.switch_to_alert() - 경고창을 띄움   

In [161]:
driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')
driver.get("http://comic.naver.com/webtoon/detail.nhn?titleld=644122&no=10")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#실행 결과는 url창이 뜨고 스크롤이 아래로 내려가짐

In [206]:
#이걸 실행하면 계속 새 창이 열리는 데 코드를 진행시키려면 
#driver 코드를 실행한 뒤 여러 창을 띄워보고 주석 처리 후 다시 실행해보기
driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')
main_window = driver.window_handles[0]  #.window_handles[0]= 첫번째 탭(메인 탭)
driver.get("http://www.google.com")

for handle in driver.window_handles:  #window_handles = 켜져있는 탭들
    if handle != main_window:  #조사중인 탭이 메인이 아니라면
        driver.switch_to.window(handle)  #handle로 탭을 옮기고
        driver.close()                   #탭을 끄는 것(메인만 남기게)
    

In [203]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.maximize_window()
location = "file:///c:/workspace/basic/alert.html"
driver.get(location)

button = driver.find_element_by_name('alert')
button.click()

obj = driver.switch_to.alert #브라우저에 실행된 alert객체 반환
msg = obj.text  #alert window의 메세지 내용
print("Alert shows following message:" + msg)

time.sleep(2)
obj.accept()   #alert window의 확인 버튼 실행
print("Clicked on the OK Button in the Alert Window")
driver.close()

Alert shows following message:Hi!, I am a Simple Alert. Please Click on the 'OK' Button.
Clicked on the OK Button in the Alert Window
